In [1]:
import requests
import time
import numpy as np
import pandas as pd
import copy

In [2]:
25*60*60

90000

In [2]:
def post_request_ytb(country_code_list, trending_type_dict):
    responses_dict = {}
    for country_code in country_code_list:
        responses_dict[country_code] = {}
        for trending_type, trending_type_url in trending_type_dict.items():
            json_data = {
                'context': {
                    'client': {
                        'gl': country_code,
                        'clientName': 'WEB',
                        'clientVersion': '2.20231115.01.01',
                        'originalUrl': 'https://www.youtube.com/feed/trending',
                    },
                },
                'browseId': 'FEtrending',
                'params': trending_type_url,
            }

            response = requests.post(
                'https://www.youtube.com/youtubei/v1/browse',
                json=json_data,
            )
            responses_dict[country_code][trending_type] = response.json()
    return responses_dict        

In [3]:
def post_request_ytb(country_code_list, trending_type_dict):
    responses_dict = {}
    for country_code in country_code_list:
        responses_dict[country_code] = {}
        for trending_type, trending_type_url in trending_type_dict.items():
            json_data = {
                'context': {
                    'client': {
                        'gl': country_code,
                        'clientName': 'WEB',
                        'clientVersion': '2.20231115.01.01',
                        'originalUrl': 'https://www.youtube.com/feed/trending',
                    },
                },
                'browseId': 'FEtrending',
                'params': trending_type_url,
            }

            response = requests.post(
                'https://www.youtube.com/youtubei/v1/browse',
                json=json_data,
            )
            responses_dict[country_code][trending_type] = response.json()
    return responses_dict        

In [4]:
def post_request_ytb(country_code_list, trending_type_dict):
    responses_dict = {}
    for country_code in country_code_list:
        responses_dict[country_code] = {}
        for trending_type, trending_type_url in trending_type_dict.items():
            json_data = {
                'context': {
                    'client': {
                        'gl': country_code,
                        'clientName': 'WEB',
                        'clientVersion': '2.20231115.01.01',
                        'originalUrl': 'https://www.youtube.com/feed/trending',
                    },
                },
                'browseId': 'FEtrending',
                'params': trending_type_url,
            }

            response = requests.post(
                'https://www.youtube.com/youtubei/v1/browse',
                json=json_data,
            )
            responses_dict[country_code][trending_type] = response.json()
    return responses_dict        

In [5]:
def collect_video_data(video_list, data_dictionary, trending_type, country_code, current_timestamp):
    # Collect data (generic videos) INPLACE
    nb_items = len(video_list)
    data_dictionary["videoId"] += [video_list[k]["videoRenderer"]["videoId"] for k in range(nb_items)]
    data_dictionary["videoType"] += [trending_type for k in range(nb_items)]
    data_dictionary["trendingCountry"] += [country_code for k in range(nb_items)]
    data_dictionary["videoTrendsRanking"] += [k for k in range(nb_items)]
    data_dictionary["scanTimestamp"] += [current_timestamp for k in range(nb_items)]
    return None

In [6]:
def collect_short_data(shorts_list, data_dictionary, country_code, current_timestamp):
    # Collect data (generic videos) INPLACE
    nb_items = len(shorts_list)
    data_dictionary["videoId"] += [shorts_list[k]["reelItemRenderer"]["videoId"] for k in range(nb_items)]
    data_dictionary["videoType"] += ['Short' for k in range(nb_items)]
    data_dictionary["trendingCountry"] += [country_code for k in range(nb_items)]
    data_dictionary["videoTrendsRanking"] += [k for k in range(nb_items)]
    data_dictionary["scanTimestamp"] += [current_timestamp for k in range(nb_items)]
    return None

In [7]:
def add_now_videos_shorts(response_dict, data_dictionary, current_timestamp):
    # Update data_dictionary with "Now", "Recently Trending" and "Shorts" videos data INPLACE
    for country_code in response_dict.keys():
        print("Current country : ", country_code)
        section_now = response_dict[country_code]['Now']["contents"]["twoColumnBrowseResultsRenderer"]["tabs"][0]["tabRenderer"]["content"]["sectionListRenderer"]["contents"]
        try:
            list_videos_before = section_now[0]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
            shorts_list = section_now[1]["itemSectionRenderer"]["contents"][0]["reelShelfRenderer"]["items"]
            list_videos_after = section_now[2]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
            list_videos_recently = section_now[3]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
        except KeyError:
            list_videos_before = section_now[1]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
            shorts_list = section_now[2]["itemSectionRenderer"]["contents"][0]["reelShelfRenderer"]["items"]
            list_videos_after = section_now[3]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
            list_videos_recently = section_now[4]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
        collect_video_data(list_videos_before, data_dictionary, 'Now', country_code, current_timestamp)
        collect_video_data(list_videos_after, data_dictionary, 'Now', country_code, current_timestamp)
        collect_video_data(list_videos_recently, data_dictionary, 'Recently Trending', country_code, current_timestamp)
        collect_short_data(shorts_list, data_dictionary, country_code, current_timestamp)
    return None

In [8]:
def add_other_sections(response_dict, data_dictionary, current_timestamp):
    # Update data_dictionary with "Music", "Gaming", "Movies" videos data INPLACE
    for country_code in response_dict.keys():
        for idx, key in enumerate(response_dict[country_code].keys()): # in recent Python versions, dictionary are ordered
            if idx != 0:
                sections = response_dict[country_code][key]["contents"]["twoColumnBrowseResultsRenderer"]["tabs"][idx]["tabRenderer"]["content"]["sectionListRenderer"]["contents"]
                video_list = sections[0]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
                collect_video_data(video_list, data_dictionary, key, country_code, current_timestamp)
    return None

In [9]:
def update_video_data(data_dictionary):
    # Update data for individual videos in data_dictionary INPLACE
    for video_id in data_dictionary["videoId"]:
        print("Currently processing video_id:", video_id)
        json_data = {
            'context': {
                'client': {
                    'clientName': 'WEB',
                    'clientVersion': '2.20231208.01.00',            
                    }
            },
            'videoId': video_id,
        }
        
        response = requests.post('https://www.youtube.com/youtubei/v1/next', json=json_data)
        video_response = response.json()
        individual_video_data = video_response["contents"]["twoColumnWatchNextResults"]["results"]["results"]["contents"]
        data_dictionary["exactViewNumber"].append(individual_video_data[0]["videoPrimaryInfoRenderer"]["viewCount"]["videoViewCountRenderer"]["viewCount"]["simpleText"])
        data_dictionary["numberLikes"].append(individual_video_data[0]["videoPrimaryInfoRenderer"]["videoActions"]["menuRenderer"]["topLevelButtons"][0]["segmentedLikeDislikeButtonViewModel"]["likeButtonViewModel"]["likeButtonViewModel"]["toggleButtonViewModel"]["toggleButtonViewModel"]["defaultButtonViewModel"]["buttonViewModel"]["title"])
        data_dictionary["creatorSubscriberNumber"].append(individual_video_data[1]["videoSecondaryInfoRenderer"]["owner"]["videoOwnerRenderer"]["subscriberCountText"]["simpleText"])     
        try:    
            data_dictionary["numberOfComments"].append(individual_video_data[2]["itemSectionRenderer"]["contents"][0]["commentsEntryPointHeaderRenderer"]["commentCount"]["simpleText"])
        except KeyError: # correct for additional "merchandise" section
            try:
                data_dictionary["numberOfComments"].append(individual_video_data[3]["itemSectionRenderer"]["contents"][0]["commentsEntryPointHeaderRenderer"]["commentCount"]["simpleText"])
            except IndexError: # comments are turned off
                data_dictionary["numberOfComments"].append(np.nan)
        try:
            data_dictionary["isCreatorVerified"].append(individual_video_data[1]["videoSecondaryInfoRenderer"]["owner"]["videoOwnerRenderer"]["badges"][0]["metadataBadgeRenderer"]["tooltip"] == "Verified")
        except KeyError:
            data_dictionary["isCreatorVerified"].append(False)    
        time.sleep(1)
    return None

In [10]:
def update_meta_data(data_dictionary):
    # Update metadata for individual videos in data_dictionary INPLACE
    for video_id in data_dictionary["videoId"]:
        print("Currently processing video_id:", video_id)
        json_data = {
        'context': {
            'client': {
                'clientName': 'WEB',
                'clientVersion': '2.20231208.01.00',
            },
        },
        'videoId': video_id,
        }
        
        response = requests.post('https://www.youtube.com/youtubei/v1/player', json=json_data)
        
        meta_response = response.json()
        try:
            data_dictionary["videoKeywords"].append(meta_response["videoDetails"]["keywords"])
        except KeyError: # no keywords
            data_dictionary["videoKeywords"].append(np.nan)
        data_dictionary["videoLengthSeconds"].append(meta_response["videoDetails"]["lengthSeconds"])
        data_dictionary["videoCategory"].append(meta_response["microformat"]["playerMicroformatRenderer"]["category"])
        data_dictionary["videoExactPublishDate"].append(meta_response["microformat"]["playerMicroformatRenderer"]["uploadDate"])
        time.sleep(1)
    return None

In [11]:
TRENDING_TYPE_DICT = {
    'Now': None,
    # 'Music': '4gINGgt5dG1hX2NoYXJ0cw%3D%3D',
    # 'Gaming': '4gIcGhpnYW1pbmdfY29ycHVzX21vc3RfcG9wdWxhcg%3D%3D',
    # 'Movie': '4gIKGgh0cmFpbGVycw%3D%3D',
}

FRENCH_COUNTRY_DICT = ['FR']

In [12]:
def scan_trending_ytb(n_scans=5, delta_scans=1800):
    """

    Args:
        n_scans (_type_): nombre de fois ou les tendances sont scannees
        delta_scans (_type_): delai entre deux scans des tendances (en secondes)
        
    """
    data_dictionary = {
    'videoId': [],
    'scanTimestamp': [],
    'videoExactPublishDate': [],
    'creatorSubscriberNumber': [],
    'videoTrendsRanking': [],
    'videoLengthSeconds': [],
    'videoType': [],
    'videoCategory': [],
    'trendingCountry': [],
    'exactViewNumber': [],
    'numberLikes': [],
    'numberOfComments': [],
    'isCreatorVerified': [],
    'videoKeywords': []
}
    data_dictionary_panel = [copy.deepcopy(data_dictionary) for k in range(n_scans)]
    Epoches=["Epoch" for k in range(n_scans)] 
    for scan in range(n_scans):
        current_timestamp = time.time()
        response_dict = post_request_ytb(FRENCH_COUNTRY_DICT, TRENDING_TYPE_DICT)
        add_now_videos_shorts(response_dict, data_dictionary_panel[scan], current_timestamp)
        add_other_sections(response_dict, data_dictionary_panel[scan], current_timestamp)
        update_video_data(data_dictionary_panel[scan])
        update_meta_data(data_dictionary_panel[scan])
        for k in data_dictionary_panel[scan].keys():
            print(len(data_dictionary_panel[scan][k]))
        time.sleep(delta_scans)
        cles=data_dictionary_panel[scan].keys()
        #cles_zero=cles[0]
        #n=data_dictionary_panel[scan][cles_zero].shape[0]
        n=112
        data_dictionary_panel[scan][Epoches[scan]]=np.full(n,scan+1)

    return pd.concat([pd.DataFrame(lst) for lst in data_dictionary_panel])

In [13]:
def pooling_scan_trend(panel_date_df):
    n_scans=len(panel_date_df)
    full_sample=pd.concat([panel_date_df[i] for i in range(n_scans)],ignore_index=False).sort_values(by=['id'])
    return full_sample

In [134]:
np.full(3,1)

array([1, 1, 1])

In [13]:
def standardize_data(arr):
    # Your code goes here
    means_row=np.mean(arr,axis=1)
    stds_row=np.std(arr,axis=1)

    return arr-means_row/stds_row

In [22]:
import numpy as np
a=np.ones((3,3))
a[:,0]=2
np.std(a,axis=1)
#a/np.array([1,1,1])
standardize_data(a)

array([[-0.82842712, -1.82842712, -1.82842712],
       [-0.82842712, -1.82842712, -1.82842712],
       [-0.82842712, -1.82842712, -1.82842712]])

In [16]:
#panel_data_df.head(2)

In [14]:
standardize_data(a)

C:\Users\DIONE Oumar\AppData\Local\Temp\ipykernel_4504\538181173.py:6: RuntimeWarning: divide by zero encountered in true_divide
  return arr-means_row/stds_row
C:\Users\DIONE Oumar\AppData\Local\Temp\ipykernel_4504\538181173.py:6: RuntimeWarning: invalid value encountered in true_divide
  return arr-means_row/stds_row


array([[ nan, -inf, -inf],
       [ nan, -inf, -inf],
       [ nan, -inf, -inf]])

In [ ]:
panel_data_df_final = scan_trending_ytb(n_scans=100,delta_scans=90000)

Current country :  FR
Currently processing video_id: bfmUsDJjBkU
Currently processing video_id: Ap7qPHeZcgM
Currently processing video_id: vvnlLGFv2mI
Currently processing video_id: usTqmg0xNZY
Currently processing video_id: n4hyVsz7FMA
Currently processing video_id: 6f1Bg-fqBRg
Currently processing video_id: 6X1Ko_TExuM
Currently processing video_id: SG065LnQ08Q
Currently processing video_id: 9RhWXPcKBI8
Currently processing video_id: VvidY6nHthA
Currently processing video_id: 6oLB9vDNvI0
Currently processing video_id: jply2m0IoYs
Currently processing video_id: OBgUnwkPivg
Currently processing video_id: BOQ1FdoBBmI
Currently processing video_id: HCJ2WHNpQqc
Currently processing video_id: Yr4wNWfEVBM
Currently processing video_id: oByOOqDerrM
Currently processing video_id: EUtsTZK0YuY
Currently processing video_id: 7bgQSVKuYdE
Currently processing video_id: J4W-oAyFvSo
Currently processing video_id: mWacdcatC9o
Currently processing video_id: TAu19lI1LMg
Currently processing video_id: 5

In [ ]:
# FIN

In [ ]:
panel_data_df_final.head()

In [71]:
panel_data_df.columns

Index(['videoId', 'scanTimestamp', 'videoExactPublishDate',
       'creatorSubscriberNumber', 'videoTrendsRanking', 'videoLengthSeconds',
       'videoType', 'videoCategory', 'trendingCountry', 'exactViewNumber',
       'numberLikes', 'numberOfComments', 'isCreatorVerified', 'videoKeywords',
       'Epoch'],
      dtype='object')

In [72]:
panel_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   videoId                  112 non-null    object 
 1   scanTimestamp            112 non-null    float64
 2   videoExactPublishDate    112 non-null    object 
 3   creatorSubscriberNumber  112 non-null    object 
 4   videoTrendsRanking       112 non-null    int64  
 5   videoLengthSeconds       112 non-null    object 
 6   videoType                112 non-null    object 
 7   videoCategory            112 non-null    object 
 8   trendingCountry          112 non-null    object 
 9   exactViewNumber          112 non-null    object 
 10  numberLikes              112 non-null    object 
 11  numberOfComments         110 non-null    object 
 12  isCreatorVerified        112 non-null    bool   
 13  videoKeywords            79 non-null     object 
 14  Epoch                    1

In [73]:
panel_data_df.head(3)

,videoId,scanTimestamp,videoExactPublishDate,creatorSubscriberNumber,videoTrendsRanking,videoLengthSeconds,videoType,videoCategory,trendingCountry,exactViewNumber,numberLikes,numberOfComments,isCreatorVerified,videoKeywords,Epoch
0,usTqmg0xNZY,1.702825e+09,2023-12-15T08:22:26-08:00,7.23M subscribers,0,2246,Now,Entertainment,FR,"1,432,975 views",142K,4.1K,True,"[mcfly, carlito, golden moustache, fatshow, ba...",1
1,6oLB9vDNvI0,1.702825e+09,2023-12-15T09:47:00-08:00,2.1M subscribers,1,394,Now,Entertainment,FR,"513,659 views",14K,514,True,"[booskap, booska-p, rap, hiphop, interview, fr...",1
2,9RhWXPcKBI8,1.702825e+09,2023-12-16T09:00:01-08:00,219M subscribers,0,1627,Now,Entertainment,FR,"43,132,281 views",2.4M,45K,True,NaN,1


In [120]:
15*60

900

In [ ]:
panel_data_df_final = scan_trending_ytb(n_scans=100,delta_scans=15)

In [ ]:
panel_data_df = scan_trending_ytb(n_scans=1,delta_scans=30)

In [74]:
panel_data_df2 = scan_trending_ytb(n_scans=2,delta_scans=5)

Current country :  FR
Currently processing video_id: usTqmg0xNZY
Currently processing video_id: 9RhWXPcKBI8
Currently processing video_id: 6oLB9vDNvI0
Currently processing video_id: OBgUnwkPivg
Currently processing video_id: BOQ1FdoBBmI
Currently processing video_id: HCJ2WHNpQqc
Currently processing video_id: Yr4wNWfEVBM
Currently processing video_id: 5jrMwhmYW1A
Currently processing video_id: TAu19lI1LMg
Currently processing video_id: sQfMhglH4_g
Currently processing video_id: oByOOqDerrM
Currently processing video_id: EUtsTZK0YuY
Currently processing video_id: jclw_cAtcIg
Currently processing video_id: TAKjuokhpyg
Currently processing video_id: CsypIF2PxhU
Currently processing video_id: iKvxt5kpqK4
Currently processing video_id: n_Dd7h_7cOM
Currently processing video_id: 0VRHcvALBTk
Currently processing video_id: YG3EhWlBaoI
Currently processing video_id: -AwEiuiOm7I
Currently processing video_id: yqe_CvveRgU
Currently processing video_id: ULQuu2fU_3w
Currently processing video_id: f

In [93]:
panel_data_df2.columns

Index(['videoId', 'scanTimestamp', 'videoExactPublishDate',
       'creatorSubscriberNumber', 'videoTrendsRanking', 'videoLengthSeconds',
       'videoType', 'videoCategory', 'trendingCountry', 'exactViewNumber',
       'numberLikes', 'numberOfComments', 'isCreatorVerified', 'videoKeywords',
       'Epoch'],
      dtype='object')

In [95]:
panel_data_df2[['videoId','Epoch']].groupby('videoId').min()

,Epoch
videoId,
-AwEiuiOm7I,1
0VRHcvALBTk,1
17L_OB1HVVk,1
36zQmvL_2YA,1
3mHwQzN2Po4,1
...,...
utDb3IB89iU,1
wD3h9ZAa6sc,1
yXuEz2Az0Pw,1


In [92]:
panel_data_df2.head(2)

,videoId,scanTimestamp,videoExactPublishDate,creatorSubscriberNumber,videoTrendsRanking,videoLengthSeconds,videoType,videoCategory,trendingCountry,exactViewNumber,numberLikes,numberOfComments,isCreatorVerified,videoKeywords,Epoch
0,usTqmg0xNZY,1.702826e+09,2023-12-15T08:22:26-08:00,7.23M subscribers,0,2246,Now,Entertainment,FR,"1,434,780 views",142K,4.1K,True,"[mcfly, carlito, golden moustache, fatshow, ba...",1
1,9RhWXPcKBI8,1.702826e+09,2023-12-16T09:00:01-08:00,219M subscribers,1,1627,Now,Entertainment,FR,"43,200,400 views",2.4M,45K,True,NaN,1


In [90]:
video_first_occurence(panel_data_df2)

,a_i
videoId,
-AwEiuiOm7I,1
0VRHcvALBTk,1
17L_OB1HVVk,1
36zQmvL_2YA,1
3mHwQzN2Po4,1
...,...
utDb3IB89iU,1
wD3h9ZAa6sc,1
yXuEz2Az0Pw,1


In [103]:
# cette fonction , nous permettra d'avoir l'ensemble de l'ensmble des videos entrées en tendances au moins aprés le premier scar
def video_first_occurence(pooled_trend_video):
    sample_group=pooled_trend_video[['videoId','Epoch']].groupby(['videoId']).min()
    #sample_group=sample_group[sample_group.Epoch>1]
    sample_group.columns=["a_i"]
    return sample_group


In [104]:
# cette fonction retourne le datframe des vidéos entrés en tendances à la dernére scraf
def last_scraf(pooled_trend_video):
        last_epoch=pooled_trend_video['Epoch'].max()
        last_scaft=pooled_trend_video[pooled_trend_video['Epoch']==last_epoch]
        return last_scaft

In [101]:
last_scraf(panel_data_df2).Epoch

0      2
1      2
2      2
3      2
4      2
      ..
107    2
108    2
109    2
110    2
111    2
Name: Epoch, Length: 112, dtype: int32

In [112]:
def final_df(panel_data_df2):
    df_first_occur=video_first_occurence(panel_data_df2)
    last_scaft_df=last_scraf(panel_data_df2)
    df=last_scaft_df.merge(df_first_occur,how='inner',on='videoId')
    return df

In [113]:
df=final_df(panel_data_df2)
df.head(2)

,videoId,scanTimestamp,videoExactPublishDate,creatorSubscriberNumber,videoTrendsRanking,videoLengthSeconds,videoType,videoCategory,trendingCountry,exactViewNumber,numberLikes,numberOfComments,isCreatorVerified,videoKeywords,Epoch,a_i
0,usTqmg0xNZY,1.702826e+09,2023-12-15T08:22:26-08:00,7.23M subscribers,0,2246,Now,Entertainment,FR,"1,436,517 views",142K,4.1K,True,"[mcfly, carlito, golden moustache, fatshow, ba...",2,1
1,9RhWXPcKBI8,1.702826e+09,2023-12-16T09:00:01-08:00,219M subscribers,1,1627,Now,Entertainment,FR,"43,340,075 views",2.4M,45K,True,NaN,2,1


In [110]:
1+1

2

In [79]:
panel_data_df2.to_csv('panel_data_df2.csv')


In [82]:
panel_data_df2.to_csv('C:/Users/DIONE Oumar/Desktop\DSSA/infrastructures/YouTrend/demos/panel_data_df2.csv')


In [86]:
df2=pd.read_csv('panel_data_df2.csv')